In [14]:
import numpy as np
import pandas as pd
from sklearn import model_selection as cv
from sklearn.metrics.pairwise import pairwise_distances
import re

import networkx as nx
from networkx.algorithms import bipartite

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from tqdm import tqdm

import json

#### Load data

In [3]:
BASE_FOLDER = "ml-1m/"

In [15]:
ratings_df = pd.read_csv(f'ratings.dat',
                         delimiter='::', engine='python', header=None,
                         names=['user_id', 'movie_id', 'rating', 'time'])
ratings_df.head()

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [16]:
movies_df = pd.read_csv(f'movies.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['MovieID', 'Title', 'Genre'], encoding="ISO-8859-1")
movies_df.head()

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
n_users = ratings_df.user_id.max()
n_items = min(ratings_df.movie_id.max(), movies_df.MovieID.max())

In [18]:
users = np.arange(n_users)
items = np.arange(n_items)

#### Split data

In [19]:
rating_train, rating_test = cv.train_test_split(ratings_df, test_size=0.25, random_state=42)
rating_train

,user_id,movie_id,rating,time
610738,3704,3784,3,966283775
324752,1924,802,3,974694998
808217,4837,1387,4,962891248
133807,867,1196,4,975277040
431857,2631,3072,5,973613453
...,...,...,...,...
259178,1586,1077,5,974735719
365838,2129,2700,5,974643199
131932,854,3102,3,975355597
671155,4033,3479,5,965525805


In [20]:
user_item_matrix = np.zeros((n_users, n_items))
for row in rating_train.itertuples():
    if row[2] >= n_items:
        continue
    user_item_matrix[row[1]-1, row[2]-1] = row[3]

In [21]:
user_item_matrix.shape

(6040, 3952)

#### user item similarity

In [22]:
item_similarity = np.ones((n_items, n_items)) - pairwise_distances(user_item_matrix.T, metric='cosine')

In [23]:
item_similarity.shape

(3952, 3952)

#### genre similarity

- genres are pipe-separated and are selected from the following genres:
- Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western

In [24]:
genres = "Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western"
genres = genres.split("|")

In [25]:
genres_df = pd.DataFrame(0, index=np.arange(n_items), columns=genres)

for i in range(len(movies_df)):
    theGenres = movies_df.loc[i, "Genre"].split("|")
    m_id = movies_df.loc[i, "MovieID"]
    for g in theGenres:
        genres_df.loc[m_id-1, g] = 1

genres_df

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3948,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3949,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3950,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [26]:
genre_similarity = np.ones((n_items, n_items)) - pairwise_distances(genres_df, metric='cosine')

In [27]:
genre_similarity.shape

(3952, 3952)

#### release year similarity

In [28]:
movies_df['release_year'] = 0
for i in range(len(movies_df)):
    movies_df.loc[i,"release_year"] = re.findall(r"\(\d+\)", movies_df.loc[i, 'Title'])[0][1:-1]

In [29]:
movies_df['release_year'].unique()

array(['1995', '1994', '1996', '1976', '1993', '1992', '1988', '1967',
       '1964', '1977', '1965', '1982', '1962', '1990', '1991', '1989',
       '1937', '1940', '1969', '1981', '1973', '1970', '1960', '1955',
       '1956', '1959', '1968', '1980', '1975', '1986', '1948', '1943',
       '1963', '1950', '1946', '1987', '1997', '1974', '1958', '1949',
       '1972', '1998', '1933', '1952', '1951', '1957', '1961', '1954',
       '1934', '1944', '1942', '1941', '1953', '1939', '1947', '1945',
       '1938', '1935', '1936', '1926', '1932', '1930', '1971', '1979',
       '1966', '1978', '1985', '1983', '1984', '1931', '1922', '1927',
       '1929', '1928', '1925', '1923', '1999', '1919', '2000', '1920',
       '1921'], dtype=object)

In [30]:
movies_df[movies_df['release_year'] == "1995"]['MovieID']

0          1
1          2
2          3
3          4
4          5
        ... 
2749    2818
2977    3046
3377    3446
3408    3477
3504    3573
Name: MovieID, Length: 342, dtype: int64

In [31]:
year_similarity = np.zeros((n_items, n_items))
year_lyst = movies_df['release_year'].unique()

for y in year_lyst:
    id_lyst = movies_df[movies_df['release_year'] == y]['MovieID']
    for i in id_lyst:
        for j in id_lyst:
            year_similarity[i-1][j-1] = 1

In [32]:
year_similarity.shape

(3952, 3952)

#### bipartite similarity

- add node and edges

In [25]:
users_str = np.array([str(user) for user in users])
ratings_df['user_id_str'] = ratings_df['user_id'].apply(str)

In [26]:
g = nx.Graph()
g.add_nodes_from(users_str, bipartite=0)
g.add_nodes_from(items, bipartite=1)

edges = ratings_df[['user_id_str', 'movie_id']].values
g.add_edges_from(edges)

In [27]:
B = bipartite.overlap_weighted_projected_graph(g, items)
B_edges = list(B.edges(data=True))

In [28]:
B_similarity = np.zeros((n_items, n_items))
for i, j, weight in B_edges:
    B_similarity[i-1, j-1] = weight['weight']
    B_similarity[j-1, i-1] = weight['weight']

In [29]:
B_similarity.shape

(3952, 3952)

In [30]:
pd.DataFrame(B_similarity).to_csv("B_similarity.csv")

In [33]:
B_similarity = pd.read_csv("B_similarity.csv")
B_similarity = np.array(B_similarity.drop('Unnamed: 0', axis=1))

#### transition matrix

In [34]:
transition_matrix_item = np.zeros((n_items, n_items))
transition_matrix_B = np.zeros((n_items, n_items))
transition_matrix_year = np.zeros((n_items, n_items))
transition_matrix_genre = np.zeros((n_items, n_items))

for i in range(n_items):
    sim_sum = item_similarity[i].sum()
    genre_sum = genre_similarity[i].sum()
    year_sum = year_similarity[i].sum()
    b_sum = B_similarity[i].sum()
    
    for j in range(n_items):
        transition_matrix_item[i,j] = item_similarity[i][j]/sim_sum
        transition_matrix_genre[i,j] = genre_similarity[i][j]/genre_sum
        transition_matrix_year[i,j] = year_similarity[i][j]/year_sum
        transition_matrix_B[i,j] = B_similarity[i][j]/b_sum
        

/var/folders/lw/89cmv85n17z5b_mb4hj4l5z80000gn/T/ipykernel_52046/670482545.py:16: RuntimeWarning: invalid value encountered in double_scalars
  transition_matrix_B[i,j] = B_similarity[i][j]/b_sum
/var/folders/lw/89cmv85n17z5b_mb4hj4l5z80000gn/T/ipykernel_52046/670482545.py:14: RuntimeWarning: invalid value encountered in double_scalars
  transition_matrix_genre[i,j] = genre_similarity[i][j]/genre_sum
/var/folders/lw/89cmv85n17z5b_mb4hj4l5z80000gn/T/ipykernel_52046/670482545.py:15: RuntimeWarning: invalid value encountered in double_scalars
  transition_matrix_year[i,j] = year_similarity[i][j]/year_sum


In [35]:
test_user_item_matrix = np.zeros((n_users, n_items))
for row in rating_test.itertuples():
    if row[2] >= n_items:
        continue
    test_user_item_matrix[row[1]-1, row[2]-1] = row[3]

In [36]:
test_user_item_matrix.shape

(6040, 3952)

In [37]:
scaler = MinMaxScaler()

def random_walk_predict(ratings, transition_matrix, alpha):
    x = (np.eye(n_items)-alpha*transition_matrix)
    
    #Obtain mean of columns as you need, nanmean is convenient.
    col_mean = np.nanmean(x, axis=0)
    #Find indices that you need to replace
    inds = np.where(np.isnan(x))
    #Place column means in the indices. Align the arrays using take
    x[inds] = np.take(col_mean, inds[1])
    
    p_tilde = np.linalg.pinv(x)
    final_rating = alpha*np.dot(ratings, np.dot(x, p_tilde))
    scaled_prediction = (scaler.fit_transform(final_rating.T)*5).T
    return scaled_prediction

In [38]:
def rmse(prediction, test_data_matrix):
    mask = test_data_matrix.nonzero()
    prediction = prediction[mask].flatten()
    truth = test_data_matrix[mask].flatten()
    return sqrt(mean_squared_error(prediction, truth))

In [39]:
alphas = [0.7, 0.8, 0.9, 1.0]
betas = [0.6, 0.7, 0.8, 0.9, 1.0]
w1s = w2s = w3s = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
weights = [(w1, w2, w3) for w1 in w1s for w2 in w2s for w3 in w3s if w1 + w2 +w3 <= 1 ]
parameters = [(alpha, beta, w1, w2, w3) for alpha in alphas for beta in betas for w1, w2 ,w3 in weights]

In [40]:
len(parameters)

5720

In [41]:
def run_grid_search(parameters):
    results = {}
    for parameter in tqdm(parameters):
        alpha, beta, w1, w2, w3 = parameter
        transition_matrix = beta*(w1*transition_matrix_item + w2*transition_matrix_year + w3*transition_matrix_B + (1-w1-w2-w3)*transition_matrix_genre) + (1-beta)/n_items
        try:
            prediction = random_walk_predict(user_item_matrix, transition_matrix, alpha)
        except:
            continue
        results[parameter] = rmse(prediction, test_user_item_matrix)
    return results

In [43]:
run_grid_search([(0.9, 0.7, 0.0, 0.4, 0.6)])

100%|██████████| 1/1 [00:31<00:00, 31.94s/it]


{(0.9, 0.7, 0.0, 0.4, 0.6): 3.029640112176484}

In [42]:
results = run_grid_search(parameters)

  0%|          | 2/5720 [00:53<42:50:01, 26.97s/it]

In [106]:
alpha, beta, w1, w2, w3 = min(results, key=results.get)
alpha, beta, w1, w2, w3

(0.9, 0.7, 0.0, 0.4, 0.6)

In [113]:
pd.DataFrame({"key":results.keys(), "values":results.values()}).to_csv("results.csv", index=False)